# Parameters Values

## Importing Libraries

In [2]:
import requests         # Used to make HTTP requests to the Redash API.
import pandas as pd     # Provides data structures and data analysis tools (e.g., DataFrame).
import smtplib          # Enables sending emails using the Simple Mail Transfer Protocol (SMTP).
from email.mime.multipart import MIMEMultipart   # For creating multipart MIME messages.
from email.mime.text import MIMEText             # For creating MIME objects of major type text.
import time
from time import sleep  # Allows us to pause execution of the script for a specified amount of time.
import io
import numpy as np
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
import smtplib

## API Configuration

In [3]:
# Redash API configuration
api_url = 'http://10.20.32.6:5000/api' # The base URL where the Redash server is accessible.
api_key = 'fpSflr67pifLS1WVyhOtyciUeeRVcp7t8zfZnx3D'  # API key for authenticating the requests.
query_ids = [53, 54, 55, 56]  # List of Query IDs that you want to execute and fetch data from.


### With these configurations in place, you can proceed to write the functions or scripts to interact with the Redash API, process the fetched data, and send it through email as needed. Each section's functionality can be encapsulated in functions for better modularity and reusability of the code.

## Preceeding towards execution of Queries

## Complete Script to Execute and Retrieve Data from Redash

### Step 1: Define Function to Execute and Refresh a Query
This function sends a POST request to trigger the execution of a Redash query. This is necessary when you want to ensure that the data you retrieve is the most recent data available.

In [5]:
def execute_redash_query(query_id):
    """Initiate execution of a Redash query and return the job information."""
    execution_url = f"{api_url}/queries/{query_id}/refresh"
    headers = {
        'Authorization': f"Key {api_key}",
        'Content-Type': 'application/json'
    }
    response = requests.post(execution_url, headers=headers)
    if response.status_code == 200:
        job = response.json()['job']
        print(f"Query {query_id} execution triggered successfully, job ID: {job['id']}")
        return job
    else:
        print(f"Failed to trigger execution for query {query_id}. Status code: {response.status_code}")
        print(f"Error response: {response.text}")
        return None

In [6]:
def check_query_status(job_id):
    """Poll the status of a query execution until completion."""
    status_url = f"{api_url}/jobs/{job_id}"
    headers = {'Authorization': f'Key {api_key}'}
    while True:
        response = requests.get(status_url, headers=headers)
        if response.status_code == 200:
            status = response.json()['job']['status']
            if status == 3:  # Status 3 means the query execution has completed successfully
                print("Query execution completed successfully.")
                return True
            elif status == 4:  # Status 4 means the query execution has failed
                print("Query execution failed.")
                return False
        else:
            print(f"Failed to check status for job {job_id}. Status code: {response.status_code}")
            return False
        time.sleep(5)  # Wait for a short period before polling again

### Step 2: Define Function to Retrieve Query Results
After triggering the query execution, this function fetches the results. It's important to ensure the query has finished executing before calling this function.

In [7]:
def fetch_query_results(query_id):
    """Fetch the results of a completed query execution."""
    headers = {'Authorization': f'Key {api_key}'}
    results_url = f"{api_url}/queries/{query_id}/results.json"
    response = requests.get(results_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['query_result']['data']['rows'])
        print(f"Results fetched successfully for query ID {query_id}.")
        return df
    else:
        print(f"Failed to fetch results for query ID {query_id}. Status code: {response.status_code}")
        print(f"Error response: {response.text}")
        raise Exception(f"Failed to fetch results. Response: {response.text}")

### Fetching and Assigning
Define Query IDs: A list named query_ids is created containing four integers: 53, 54, 55, and 56. These numbers represent specific query identifiers that will be used to fetch data from a database or a service like Redash.

Initialize an Empty Dictionary: A dictionary named dfs is initialized to store the results of each query. Each query's results will later be assigned to this dictionary with keys formatted as 'df' followed by the query ID (e.g., 'df53').

Loop Over Query IDs: A for loop is initiated to process each query ID in the query_ids list one by one.
Print Status Message: For each query ID, a message is printed to indicate that the processing of that particular query ID is starting.

Execute Query: The function execute_redash_query(query_id) is called with the current query ID. This function is responsible for triggering the execution of the query on the Redash server.

Wait for Completion: The code then pauses for 10 seconds using sleep(10). This delay is intended to give enough time for the query to complete its execution on the server before the results are fetched.

Fetch Query Results: After the wait, the fetch_query_results(query_id) function is called to retrieve the results of the executed query and store them in the variable df.

Assign DataFrames to Specific Variables: In the final step, the DataFrames stored in the dictionary are unpacked into individual variables df53, df54, df55, and df56. This is done using a generator expression (dfs[f'df{query_id}'] for query_id in query_ids) which iterates over the same list of query IDs and fetches each corresponding DataFrame from the dfs dictionary.

In [8]:
# Main execution loop
dfs = {}
for query_id in query_ids:
    print(f"Processing query ID {query_id}...")
    job_info = execute_redash_query(query_id)
    if job_info and check_query_status(job_info['id']):
        df = fetch_query_results(query_id)
        dfs[f'df{query_id}'] = df

# Assign DataFrames to explicit variables
df53, df54, df55, df56 = (dfs[f'df{query_id}'] for query_id in query_ids)

Processing query ID 53...
Query 53 execution triggered successfully, job ID: e87ffd42-3f09-48df-b429-87eefc270248
Query execution completed successfully.
Results fetched successfully for query ID 53.
Processing query ID 54...
Query 54 execution triggered successfully, job ID: fa662ca7-d45d-47f9-8cb0-e901ea2a2b54
Query execution completed successfully.
Results fetched successfully for query ID 54.
Processing query ID 55...
Query 55 execution triggered successfully, job ID: 9fa798d3-c2f7-4efb-a334-34bd76130126
Query execution completed successfully.
Results fetched successfully for query ID 55.
Processing query ID 56...
Query 56 execution triggered successfully, job ID: 5c880e33-e7d9-4e86-9236-9df508d00059
Query execution completed successfully.
Results fetched successfully for query ID 56.


## Reading an External File to Categorize Clinic Names as Internal, External, or Test

In [9]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQW0rTtOgAX7cAXkO-eY5GB8AkPhs2Org5HvE2X-dACf0NjrHHXnuR8cVUCST7xA5luqu_8uS3KSEho/pub?output=csv'
df1 = pd.read_csv(url)

### Renaming Clinic Names to Align with Required Specifications

In [10]:
df1.rename(columns={'Clinic_Name': 'Clinic Name'}, inplace=True)
df53.rename(columns={'Clinic_Name': 'Clinic Name'}, inplace=True)
df54.rename(columns={'Clinic_Name': 'Clinic Name'}, inplace=True)
df55.rename(columns={'Clinic_Name': 'Clinic Name'}, inplace=True)
df56.rename(columns={'Clinic_Name': 'Clinic Name'}, inplace=True)

### Merging Data and Assigning the Result to a New DataFrame

Initialize the Base DataFrame: The DataFrame df2 is initialized by setting it equal to df1. This makes df2 the starting point or base DataFrame for subsequent merging operations.

Define List of DataFrames for Merging: A list named other_dfs is created containing DataFrames df53, df54, df55, and df56. These are the DataFrames that will be merged into the base DataFrame df2.

Merge DataFrames in a Loop:
The code enters a loop that iterates over each DataFrame in the other_dfs list.

Inside the loop, the pd.merge() function is used to merge df2 with the current DataFrame (df) from the list. The merging is based on the column "Clinic Name", ensuring that rows with the same clinic name in both DataFrames are aligned together.

The how="outer" parameter is specified, which means the merge will include all rows from both df2 and the current DataFrame df, even if they do not have matching entries in the "Clinic Name" column. This results in a union of all entries, filling in missing data with NaNs where no match is found.

The result of each merge is assigned back to df2, updating it with the combined data from the merge operation.

This code sequentially merges multiple DataFrames (df53, df54, df55, df56) into a base DataFrame (df1) based on matching clinic names, using an outer join to ensure all data from all DataFrames is retained. The resulting merged DataFrame (df2) contains a comprehensive dataset with entries from all involved DataFrames.

In [11]:
# Start with df1 as the base for merging
df2 = df1

# List of other DataFrames to merge
other_dfs = [df53, df54, df55, df56]

# Merge each DataFrame into df2
for df in other_dfs:
    df2 = pd.merge(df2, df, on="Clinic Name", how="outer")

### Replacing All Unknown or NaN Values with 0

In [12]:
df2.fillna(0, inplace=True)

### Removing Clinics Not Categorized as External or Internal

In [13]:
# Filter out rows where 'External / Internal' is 0
df2 = df2[df2['External / Internal'] != 0]

# Optional: Reset the index if you want a clean index after removing rows
df2.reset_index(drop=True, inplace=True)

### Rearranging and Renaming Columns According to Specified Requirements

In [14]:
# Define new column names as a dictionary
new_column_names = {
    'Clinic_Staff_Count': '#Staff Count',
    'lastLoginAt': 'Last login date',
    'loggedIn7Days': 'Last Login in past 7 days?',
    'loggedIn30Days': 'Last Login in past 30 days?',
    'Total Patients': '#Total Patients',
    'Patients added in last 30 days': '#Patients added in last 30 days',
    'Patients added in last 7 days': '#Patients added in last 7 days',
    'Meetings_Attended': '#Total meetings',
    'Meetings_Completed': '#Meetings Completed',
    '%Completion_of_Meetings': '%Completion Rate',
    'Proprietary_Supplements_Count': '#Proprietary Supplements Count'
}

# Rename the columns
df2.rename(columns=new_column_names, inplace=True)

# Define the new order of the columns, excluding the removed column
column_order = [
    'Clinic Name',
    'External / Internal',
    '#Staff Count',
    'Last login date',
    'Last Login in past 7 days?',
    'Last Login in past 30 days?',
    '#Total Patients',
    '#Patients added in last 30 days',
    '#Patients added in last 7 days',
    '#Total meetings',
    '#Meetings Completed',
    '%Completion Rate',
    '#Proprietary Supplements Count'
]

# Reorder the columns in df2
df2 = df2[column_order]

### Replacing All Unknown or NaN Values with 0

In [15]:
# Columns to update
columns_to_update = ['Last login date', 'Last Login in past 7 days?', 'Last Login in past 30 days?']

# Replace '0' with '-' in these columns
df2[columns_to_update] = df2[columns_to_update].replace({0: '-'})

### Creating and processing a pivot table in pandas:

1. **Import Libraries**:
   - Import `numpy` and `pandas` libraries, which are fundamental for data manipulation in Python. `numpy` is typically used for numerical operations.

2. **Creating the Pivot Table**:
   - `df3 = df2.pivot_table(...)`: Create a pivot table from `df2`. A pivot table is a summarized table used for data analysis.
   - `index='External / Internal'`: Set the index of the pivot table to the 'External / Internal' column to group data based on these categories.
   - `values=[...]`: Define the columns to be included in the pivot table. These columns will have aggregation functions applied as defined in `aggfunc`.
   - `aggfunc={...}`: Assign specific aggregation functions to each column:
     - `'Clinic Name': pd.Series.nunique`: Count unique values in the 'Clinic Name' column.
     - All other specified columns (`'#Staff'`, `'#Patients'`, etc.): Sum up the values using `np.sum`.
   - `margins=True`: Include a 'Grand Total' row at the end of the pivot table that provides the overall sum for each column.
   - `margins_name='Grand Total'`: Set the name for the total row.

3. **Calculate %Completion of Meetings**:
   - `df3['%Completion of Meetings']`: Calculate the percentage of meetings completed by dividing the '#Meetings Completed' by '#Total meetings', replacing NaN values with 0.
   - `.apply(lambda x: f"{x:.0%}")`: Format the result as a percentage with no decimal places.

4. **Rename Columns**:
   - `df3.rename(columns={'Clinic Name': '#Clinic Count'}, inplace=True)`: Rename the 'Clinic Name' column in `df3` to '#Clinic Count' to better reflect that it represents a count of unique clinic names.

5. **Ensure Correct Column Order**:
   - `column_order = [...]`: Define a specific order for the columns.
   - `df3 = df3[column_order]`: Reorder the columns in `df3` according to the list `column_order` to ensure the data is presented in a logical and consistent format.

**Summary**: The code constructs a pivot table from an existing DataFrame, applies specific aggregation functions, calculates additional metrics, and formats the table with precise naming and ordering to enhance readability and utility for analysis.

In [16]:
# Creating the pivot table
df3 = df2.pivot_table(
    index='External / Internal',
    values=[
        'Clinic Name', '#Staff Count', '#Patients added in last 30 days', '#Patients added in last 7 days',
        '#Total Patients', '#Total meetings', '#Meetings Completed', '#Proprietary Supplements Count'
    ],
    aggfunc={
        'Clinic Name': pd.Series.nunique,  # Counts unique clinic names as '#Clinic Count'
        '#Staff Count': np.sum,
        '#Patients added in last 30 days': np.sum,
        '#Patients added in last 7 days': np.sum,
        '#Total Patients': np.sum,
        '#Total meetings': np.sum,
        '#Meetings Completed': np.sum,
        '#Proprietary Supplements Count': np.sum
    },
    margins=True,
    margins_name='Grand Total'
)

# Calculate %Completion of Meetings
df3['%Completion of Meetings'] = (df3['#Meetings Completed'] / df3['#Total meetings']).fillna(0).apply(lambda x: f"{x:.0%}")

# Rename the 'Clinic Name' to '#Clinic Count'
df3.rename(columns={'Clinic Name': '#Clinic Count'}, inplace=True)

# Ensure the correct column order before resetting index
column_order = [
    '#Clinic Count', '#Staff Count', '#Patients added in last 30 days', 
    '#Patients added in last 7 days', '#Total Patients', '#Total meetings', 
    '#Meetings Completed', '%Completion of Meetings', '#Proprietary Supplements Count'
]
df3 = df3[column_order]

# Reset the index to turn the 'External / Internal' index into a regular column
df3.reset_index(inplace=True)

# Rename the columns appropriately
df3.rename(columns={'External / Internal': 'Affiliate Category'}, inplace=True)

# Reorder the DataFrame to ensure 'Affiliate Category' is the first column
df3 = df3[['Affiliate Category', '#Clinic Count', '#Staff Count', '#Proprietary Supplements Count', 
           '#Total Patients', '#Patients added in last 30 days', '#Patients added in last 7 days', 
           '#Total meetings', '#Meetings Completed', '%Completion of Meetings']]

# Convert numerical columns to integers to remove the decimal points
numeric_columns = ['#Clinic Count', '#Staff Count', '#Proprietary Supplements Count', '#Total Patients', 
                   '#Patients added in last 30 days', '#Patients added in last 7 days', 
                   '#Total meetings', '#Meetings Completed']
df3[numeric_columns] = df3[numeric_columns].astype(int)

# Display the updated DataFrame
df3.head()

C:\Users\asus\AppData\Local\Temp\ipykernel_4048\1405056759.py:2: FutureWarning: The provided callable <function sum at 0x000002767FA55EE0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df3 = df2.pivot_table(


,Affiliate Category,#Clinic Count,#Staff Count,#Proprietary Supplements Count,#Total Patients,#Patients added in last 30 days,#Patients added in last 7 days,#Total meetings,#Meetings Completed,%Completion of Meetings
0,External,15,22,2,87,4,0,42,3,7%
1,Internal Healthcare Team,25,35,668,782,76,22,628,196,31%
2,Non-Healthcare (Tech / Test accounts),20,151,153,402,76,20,800,273,34%
3,Grand Total,60,208,823,1271,156,42,1470,472,32%


### Modifying df3

1. **Resetting the Index**:
   - `df3.reset_index(inplace=True)`: This line converts the index of the DataFrame `df3` into a regular column. Since 'External / Internal' is set as an index, this action transforms it back into a standard column within the DataFrame. The `inplace=True` parameter modifies the DataFrame directly, saving the need to reassign it.

2. **Renaming Columns**:
   - `df3.rename(columns={'External / Internal': 'Affiliate Type - External / Internal'}, inplace=True)`: This changes the name of the 'External / Internal' column to 'Affiliate Type - External / Internal' for clearer identification. Again, `inplace=True` ensures the DataFrame is updated directly.

3. **Dropping Unwanted Rows**:
   - This step involves conditional logic to remove unwanted rows based on specific criteria. The condition `if df3.iloc[1, 0] == 'Unwanted Row Identifier'` checks if the first column of the second row (indexed at 1) matches 'Unwanted Row Identifier'. If it does, that row is removed using `df3.drop(index=1, inplace=True)`. This function deletes the row at index 1 and modifies the DataFrame in place.

4. **Reordering the DataFrame**:
   - The last line rearranges the columns of `df3` into a specified order, ensuring that 'Affiliate Type - External / Internal' appears first. The list provided in the brackets specifies the new order of the columns, and the DataFrame is reassigned to `df3` with these columns reordered accordingly.

**Summary**: The code modifies the DataFrame `df3` by resetting its index, renaming a column for clarity, optionally removing an unwanted row based on a specific condition, and reordering the columns to ensure logical presentation. These steps are typically used to clean and organize data for better analysis and reporting.

In [17]:
# Filter the DataFrame for external clinics and select relevant columns
df56a = df2[df2['External / Internal'] == 'External'][[
    'Clinic Name', 'External / Internal', 'Last login date', 
    'Last Login in past 7 days?', 'Last Login in past 30 days?'
]]

# Rename 'Clinic Name' to 'Affiliate Name'
df56a.rename(columns={'Clinic Name': 'Affiliate Name'}, inplace=True)

# Modify 'Last login date' to display only the date
df56a['Last login date'] = pd.to_datetime(df56a['Last login date'], errors='coerce').dt.date

# Replace NaT values with '-'
df56a['Last login date'] = df56a['Last login date'].apply(lambda x: '-' if pd.isna(x) else x)

### Email creation and sending with HTML content and CSV attachments:

1. **Define Conversion Function**:
   - `dataframe_to_email_attachment(df, filename)`: Defines a function to convert a pandas DataFrame into a MIMEApplication object suitable for email attachments. It:
     - Creates a CSV from the DataFrame using `StringIO`, which acts like a file held in memory.
     - Sets the file's name within the MIME object and its content disposition, facilitating its use as an attachment in emails.
   
2. **Convert DataFrames to HTML**:
   - `html_content_df3 = df3.to_html(index=False)`: Converts the DataFrame `df3` to HTML format for embedding directly into the email. `index=False` prevents the index from being included in the HTML table.
   - `html_content_df56 = df56.to_html(index=False)`: Similar conversion for DataFrame `df56`.

3. **Combine HTML Content**:
   - `combined_html`: Combines the HTML representations of the two DataFrames with headings into a single HTML string. This will be the body of the email.

4. **Create Filename and Attachments**:
   - `today_date`: Gets the current date formatted as 'YYYY-MM-DD'.
   - `filename_df3` and `filename_df56`: Constructs filenames for the CSV files incorporating the date.
   - `attachment_df3` and `attachment_df56`: Converts `df3` and `df56` to MIMEApplication objects using the previously defined function.

5. **Setup Email**:
   - `message = MIMEMultipart()`: Starts assembling a multipart email message that can include both HTML content and file attachments.
   - Sets the 'From', 'To', and 'Subject' fields of the email.
   - Attaches the combined HTML content and both CSV files to the email message.

6. **Send Email via SMTP**:
   - Establishes an SSL connection to the SMTP server on the given port.
   - Logs in to the server using the provided email credentials.
   - Sends the prepared email message.
   - Catches and prints any errors during the sending process, which helps in troubleshooting.

7. **Print Confirmation or Error Message**:
   - Outputs a confirmation if the email was sent successfully or details of any error that occurred, providing feedback on the operation's result.

**Summary**: This script automates the process of sending a detailed email report, which includes both visual (HTML) and downloadable (CSV) data summaries, by programmatically converting data frames, creating email content, attaching files, and handling SMTP email sending, all encapsulated in a structured and error-handling script.

In [18]:
# Email configuration
sender_email = "yash.kumar@lybl.com"         # The email address from which the report will be sent.
sender_password = "OnePlux@1972"             # Password for the sender's email account.
receiver_emails = "yash.kumar@lybl.com"  # List of recipient email addresses.
smtp_server = "smtp.gmail.com"               # The SMTP server through which emails will be sent.
smtp_port = 465  

In [21]:
import io
from datetime import datetime, timedelta
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

# Custom function to format date without ordinal suffix
def format_date(date):
    return date.strftime("%d %B '%y")

# Get yesterday's date and format it
yesterday_date = datetime.now() - timedelta(days=1)
formatted_yesterday_date = format_date(yesterday_date)

# Function to convert DataFrame to MIMEApplication object
def dataframe_to_email_attachment(df, filename):
    """Convert DataFrame to MIMEApplication object for email attachment."""
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)
    mime_object = MIMEApplication(csv_buffer.read(), Name=filename)
    mime_object['Content-Disposition'] = f'attachment; filename="{filename}"'
    return mime_object

# Calculate summary statistics
external_affiliates = df2[df2['External / Internal'] == 'External']
n = external_affiliates['Clinic Name'].nunique()
p = int(external_affiliates['#Staff Count'].sum())
q = int(external_affiliates['#Total Patients'].sum())
r = int(external_affiliates['#Total meetings'].sum())
s = int(external_affiliates['#Proprietary Supplements Count'].sum())
x = int(external_affiliates['#Patients added in last 30 days'].sum())
y = int(external_affiliates['#Patients added in last 7 days'].sum())

# Convert DataFrames to HTML for email content
html_content_df3 = df3.to_html(index=False, classes="dataframe")
html_content_df56a = df56a.to_html(index=False, classes="dataframe")

# Adding inline styles to ensure the first column and header row are left-aligned
def add_inline_styles(html_content):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Left-align all header cells
    for th in soup.find_all('th'):
        th['style'] = 'text-align: left;'
    
    # Left-align first column cells
    for row in soup.find_all('tr'):
        first_td = row.find('td')
        if first_td:
            first_td['style'] = 'text-align: left;'
    
    return str(soup)

html_content_df3 = add_inline_styles(html_content_df3)
html_content_df56a = add_inline_styles(html_content_df56a)

# Styling for the tables
style = """
<style>
    .dataframe {
        width: 100%;
        border-collapse: collapse;
        table-layout: fixed; /* Ensures each column has equal width */
    }
    .dataframe th, .dataframe td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center; /* Centers text by default */
    }
    .dataframe th:first-child, .dataframe td:first-child {
        text-align: left; /* Left-aligns the first column and header cells */
    }
    .dataframe thead th {
        background-color: #f2f2f2;
        color: black;
    }
    .dataframe tfoot th {
        background-color: #f9f9f9;
    }
</style>
"""

# Create attachments with formatted yesterday's date
filename_df3 = f"Production_Summary_{formatted_yesterday_date}.csv"
filename_df56a = f"External_Affiliate_Login_Details_{formatted_yesterday_date}.csv"
attachment_df3 = dataframe_to_email_attachment(df3, filename_df3)
attachment_df56a = dataframe_to_email_attachment(df56a, filename_df56a)

# Email body content setup
email_subject = f"LYBL Platform Usage Weekly Report (Affiliates) | {formatted_yesterday_date}"
email_body = f"""
<html>
<head>
{style}
</head>
<body>
<p>Hi Team,</p>
<p>Please find the latest usage analytics (affiliates) updated as on {formatted_yesterday_date}.</p>

<h3>How to read this report?</h3>
<p><b>High level platform usage</b></p>
<p>The ‘{n}’ external affiliates together have added ‘{p}’ staffs, ‘{q}’ patients, ‘{r}’ meetings and ‘{s}’ proprietary supplements.</p>
<p><b>Daily / monthly level platform usage</b></p>
<p>The ‘{n}’ external affiliates have added ‘{x}’ patients in the last 30 days and ‘{y}’ patients in the last 7 days.</p>
<p>The ‘{n}’ external affiliates login information (whether they logged in the app in the last 7 days/ 30 days) is available below.</p>

<h3>Production Data</h3>
{html_content_df3}
<h3>External Affiliate Login Details with 7 days and 30 days Info</h3>
{html_content_df56a}
<p>Please find the raw data attached as CSV.</p>
<p>Thank You,</p>
<p>Yash Kumar</p>
<p>Data Analytics Intern</p>
</body>
</html>
"""

# Setup and send the email
message = MIMEMultipart()
message['From'] = sender_email
message['To'] = receiver_emails
message['Subject'] = email_subject
message.attach(MIMEText(email_body, 'html'))  # Attach HTML content
message.attach(attachment_df3)  # Attach df3 CSV
message.attach(attachment_df56a)  # Attach df56a CSV

# Send the email using SMTP
try:
    with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
        server.login(sender_email, sender_password)
        server.send_message(message)
    print("Email sent successfully with detailed HTML content and CSV attachments.")
except Exception as e:
    print(f"An error occurred while sending the email: {e}")


Email sent successfully with detailed HTML content and CSV attachments.
